# Bot or Not? : Transformer-Based Machine-Generated Text Detection

## 👤 Team
- Mohammed Bouadjimi (041179141)

## 📌 Problem Definition
With the rise of tools like ChatGPT, distinguishing human-written from machine-generated text is crucial for journalism, education, and digital integrity. This project explores how well transformer models can detect AI-generated content across domains.

## 📊 Dataset
- **Source**: [yaful/MAGE on Hugging Face](https://huggingface.co/datasets/yaful/MAGE)
- **Format**: JSON, labeled text
- **Classes**: Human (0), Machine (1)
- **Domains**: Wikipedia, Reddit, News, etc.

## 🧠 Model
- **Base model**: RoBERTa-base
- **Framework**: Hugging Face Transformers
- **Training setup**:
  - Subsampled 10k training examples
  - 3 epochs
  - Batch size 8
  - Learning rate 2e-5

## 📈 Evaluation Results

| Metric      | Value     |
|-------------|-----------|
| Accuracy    | 68.95%    |
| F1 Score    | 58.85%    |
| Test Loss   | 2.016     |

> Note: Results are based on a small training subset due to Colab limitations.

## 🧪 Baseline Comparison
Baseline models (like Logistic Regression or MLP) were outperformed by RoBERTa. Transformer-based models offer better generalization across domains.

## 🔧 Files
- `notebook.ipynb` – full Colab code and training pipeline
- `bot_or_not_model/` – saved model and tokenizer


In [1]:
!pip install -U transformers datasets scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 79.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.


In [2]:
import pandas as pd
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score


In [3]:
dataset = load_dataset("yaful/MAGE")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/404M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/72.3M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/71.7M [00:00<?, ?B/s]

test_ood_set_gpt.csv: 0.00B [00:00, ?B/s]

test_ood_set_gpt_para.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/319071 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56792 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60743 [00:00<?, ? examples/s]

In [4]:
train_small = dataset["train"].shuffle(seed=42).select(range(10000))
val_small = dataset["validation"].shuffle(seed=42).select(range(2000))
test_small = dataset["test"].shuffle(seed=42).select(range(2000))


In [5]:
checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

train_tok = train_small.map(tokenize_function, batched=True)
val_tok = val_small.map(tokenize_function, batched=True)
test_tok = test_small.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
train_tok.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_tok.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_tok.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments

# Simplified training arguments for compatibility with older versions
training_args = TrainingArguments(
    output_dir="./results",                  # Output directory for model checkpoints
    do_train=True,                           # Enable training
    do_eval=True,                            # Enable evaluation
    learning_rate=2e-5,                      # Learning rate
    per_device_train_batch_size=8,           # Training batch size
    per_device_eval_batch_size=8,            # Evaluation batch size
    num_train_epochs=3,                      # Total number of training epochs
    weight_decay=0.01,                       # Weight decay for regularization
    logging_dir="./logs",                    # Directory for logs
    logging_steps=100                        # Log every 100 steps
)


In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Compute accuracy and F1-score from predictions and labels
def compute_metrics(eval_pred):
    logits, labels = eval_pred                     # Unpack model outputs and true labels
    preds = np.argmax(logits, axis=1)              # Get predicted labels (class with max logit)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


In [11]:
from transformers import Trainer

# Set up the Trainer for training and evaluation
trainer = Trainer(
    model=model,                         # The model you loaded (RoBERTa)
    args=training_args,                  # Training arguments
    train_dataset=train_tok,            # Training dataset
    eval_dataset=val_tok,               # Validation dataset
    tokenizer=tokenizer,                # Tokenizer (for saving/loading)
    compute_metrics=compute_metrics     # Function to compute accuracy and F1
)


/tmp/ipython-input-2344483982.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: modeyofficial (modeyofficial-algonquin-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
100,0.605300
200,0.525500
300,0.470700
400,0.463600
500,0.377700
600,0.385800
700,0.368100
800,0.371700
900,0.315600
1000,0.473700


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.1

TrainOutput(global_step=3750, training_loss=0.26908232599894205, metrics={'train_runtime': 1906.5598, 'train_samples_per_second': 15.735, 'train_steps_per_second': 1.967, 'total_flos': 3946665830400000.0, 'train_loss': 0.26908232599894205, 'epoch': 3.0})

In [13]:
trainer.evaluate(test_tok)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 2.0158143043518066,
 'eval_accuracy': 0.6895,
 'eval_f1': 0.588469184890656,
 'eval_runtime': 33.576,
 'eval_samples_per_second': 59.566,
 'eval_steps_per_second': 7.446,
 'epoch': 3.0}

In [14]:
model.save_pretrained("bot_or_not_model")
tokenizer.save_pretrained("bot_or_not_model")


('bot_or_not_model/tokenizer_config.json',
 'bot_or_not_model/special_tokens_map.json',
 'bot_or_not_model/vocab.json',
 'bot_or_not_model/merges.txt',
 'bot_or_not_model/added_tokens.json',
 'bot_or_not_model/tokenizer.json')

In [15]:
import shutil
shutil.make_archive("bot_or_not_model", 'zip', "bot_or_not_model")


'/content/bot_or_not_model.zip'